In [1]:
import requests
from PIL import Image
from pprint import pprint
import json
import pandas as pd

In [2]:
api_key = "vytvIDyAwxQXP87gn0Defwiltz4391uInF8bObyE"
url_weather = "https://api.nasa.gov/insight_weather/?"
url_images = "https://api.nasa.gov/mars-photos/api/v1/rovers/{}/photos?"
rovers = ["curiosity", "opportunity", "spirit"]
# [url_images.format(x) for x in rovers]
# API Mars Weather Docs: https://api.nasa.gov/assets/insight/InSight%20Weather%20API%20Documentation.pdf

In [3]:
payload_weather = {
    "api_key": api_key,
    "feedtype": "json"
}
payload_images = {
    "api_key": api_key
}

In [4]:
# resp = requests.get(url, params=payload)
resp = requests.get(url_weather, params=payload_weather)

In [5]:
# resp.content

In [6]:
cont = json.loads(resp.content)
cont2 = {
    key: cont[key] for key in list(cont.keys())[:-2]
}
# pprint(cont2)
cont_pd = pd.DataFrame(cont2)

In [7]:
weather = []
for key,val in cont2.items():
    pre = val["PRE"]
    del val["PRE"]
    del val["WD"]
    val = {**val, **pre}
    updated = {**{"Sol":key}, **val}
    weather.append(updated)
weather

[{'Sol': '809',
  'First_UTC': '2021-03-06T10:57:03Z',
  'Last_UTC': '2021-03-07T11:36:33Z',
  'Month_ordinal': 12,
  'Northern_season': 'late winter',
  'Season': 'winter',
  'Southern_season': 'late summer',
  'av': 727.644,
  'ct': 103645,
  'mn': 706.737,
  'mx': 744.8863},
 {'Sol': '810',
  'First_UTC': '2021-03-07T11:36:38Z',
  'Last_UTC': '2021-03-08T12:16:13Z',
  'Month_ordinal': 12,
  'Northern_season': 'late winter',
  'Season': 'winter',
  'Southern_season': 'late summer',
  'av': 728.062,
  'ct': 104975,
  'mn': 707.2886,
  'mx': 744.3631},
 {'Sol': '811',
  'First_UTC': '2021-03-08T12:16:13Z',
  'Last_UTC': '2021-03-09T12:55:48Z',
  'Month_ordinal': 12,
  'Northern_season': 'late winter',
  'Season': 'winter',
  'Southern_season': 'late summer',
  'av': 728.219,
  'ct': 149673,
  'mn': 707.2492,
  'mx': 745.7073},
 {'Sol': '812',
  'First_UTC': '2021-03-09T12:55:49Z',
  'Last_UTC': '2021-03-10T13:35:24Z',
  'Month_ordinal': 12,
  'Northern_season': 'late winter',
  'Season

In [8]:
weather_pd = pd.DataFrame(weather)
new_order = ["Sol"] + [x for x in weather_pd.columns.tolist() if x != "Sol"]
weather_pd = weather_pd[new_order]

In [9]:
weather_pd

,Sol,First_UTC,Last_UTC,Month_ordinal,Northern_season,Season,Southern_season,av,ct,mn,mx
0,809,2021-03-06T10:57:03Z,2021-03-07T11:36:33Z,12,late winter,winter,late summer,727.644,103645,706.7370,744.8863
1,810,2021-03-07T11:36:38Z,2021-03-08T12:16:13Z,12,late winter,winter,late summer,728.062,104975,707.2886,744.3631
2,811,2021-03-08T12:16:13Z,2021-03-09T12:55:48Z,12,late winter,winter,late summer,728.219,149673,707.2492,745.7073
3,812,2021-03-09T12:55:49Z,2021-03-10T13:35:24Z,12,late winter,winter,late summer,727.699,105992,706.3650,742.9495
4,813,2021-03-10T13:35:28Z,2021-03-11T14:14:59Z,12,late winter,winter,late summer,727.909,100620,706.2391,743.8608
5,814,2021-03-11T14:15:03Z,2021-03-12T14:54:34Z,12,late winter,winter,late summer,728.121,104121,706.9009,745.4649
6,815,2021-03-12T14:54:38Z,2021-03-13T08:54:34Z,12,late winter,winter,late summer,728.407,64802,708.4248,744.9279


In [ ]:
# exmp = cont_pd.iloc[0]
# download_url = "https://epic.gsfc.nasa.gov/epic-archive/jpg/{}.jpg".format(exmp["image"])

In [ ]:
# im = Image.open(requests.get(download_url, stream=True).raw)

In [ ]:
# download_url

In [ ]:
# im